### Gradient Boosting

Otro método de ensamble....


Primero importamos todo lo que vamos a necesitar

In [19]:
%matplotlib inline 
import matplotlib.pylab as plt 
import numpy as np 
from scipy import sparse
from sklearn.model_selection import train_test_split, ShuffleSplit , learning_curve, GridSearchCV
from sklearn import metrics 
from sklearn.ensemble import GradientBoostingRegressor 
from pprint import pprint 
import pandas as pd 
import seaborn

Primero leemos los datos y dividimos la muestra en entrenamiento y prueba.

In [52]:
df_train = pd.read_csv("data/train_preprocesado.csv")
#por alguna razón quedaron dos Na en BsmtCond, así los quitamos pero en realidad hay que regresar a ver qué pasó
df_train.dropna(inplace=True)
# separamos en entrenamiento y validación
train, test = train_test_split(df_train, test_size=0.2)
# creamos dataframes para las variables y el objetivo
Y_train = train['SalePrice']
X_train = train.drop(['SalePrice','Id', 'Unnamed: 0'], axis=1)
Y_test = test['SalePrice']
X_test = test.drop(['SalePrice','Id', 'Unnamed: 0'], axis=1)

Vamos a ajustar los hiperparámetros del método haciendo validación cruzada. Por convenciencia, podemos empaquetar el proceso en una función para poder reutilizarlo

In [36]:
def gbCv(X, Y, param_grid, n_jobs):
    estimator = GradientBoostingRegressor()
    cv = ShuffleSplit(n_splits=3, test_size=0.2)
    regressor = GridSearchCV(estimator=estimator, cv=cv, param_grid=param_grid, n_jobs=n_jobs, verbose = 2)
    regressor.fit(X,Y)
    return cv, regressor.best_estimator_

Ahora vamos a usar la función para estimar los hiperparámetros del método

In [37]:
param_grid = {
    'n_estimators': [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)],
    'learning_rate': [0.02, 0.05, 0.1, 0.12],
    'max_depth': [4, 6, 10, 15],
    'min_samples_leaf': [1, 2, 4],
    'max_features': [0.1, 0.3, 1.0]
}

cv, best_estimator = gbCv(X_train, Y_train, param_grid, -1)

Fitting 3 folds for each of 1440 candidates, totalling 4320 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    6.7s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   48.7s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  3.7min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed: 10.1min
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed: 21.9min
[Parallel(n_jobs=-1)]: Done 1442 tasks      | elapsed: 30.9min
[Parallel(n_jobs=-1)]: Done 1969 tasks      | elapsed: 40.5min
[Parallel(n_jobs=-1)]: Done 2576 tasks      | elapsed: 48.4min
[Parallel(n_jobs=-1)]: Done 3265 tasks      | elapsed: 55.7min
[Parallel(n_jobs=-1)]: Done 4034 tasks      | elapsed: 63.3min
[Parallel(n_jobs=-1)]: Done 4320 out of 4320 | elapsed: 65.7min finished


Podemos ver los resultados que obtuvimos

In [38]:
best_estimator.get_params()

{'alpha': 0.9,
 'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 0.02,
 'loss': 'ls',
 'max_depth': 4,
 'max_features': 0.1,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 2000,
 'n_iter_no_change': None,
 'presort': 'auto',
 'random_state': None,
 'subsample': 1.0,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

Y, por su puesto, comparar el score ($R^2$) entre las muestras de entrenamiento y velidación.

In [54]:
print("El score sobre la muestra de entrenamiento es. {:4f}".format(best_estimator.score(X_train, Y_train)))
print("El score sobre la muestra de prueba es. {:4f}".format(best_estimator.score(X_test, Y_test)))

El score sobre la muestra de entrenamiento es. 0.976762
El score sobre la muestra de prueba es. 0.978132


Ahora sí, exportamos el resultado 

In [42]:
test_df = pd.read_csv("data/test_preprocesado.csv")
X_test = test_df.drop(['Id'], axis=1)
result = best_estimator.predict(X_test)
result_df = pd.DataFrame(result)
result_df.columns = ['SalePrice']
result_df = result_df.join(test_df['Id'])
result_df['SalePrice'] = np.exp(result_df['SalePrice'])
result_df.to_csv("data/result_base2.csv", index=False)

Y guardamos el modelo

In [55]:
from joblib import dump, load
dump(best_estimator, 'data/models/baseline_GB.joblib')

['data/models/baseline_GB.joblib']